In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
# root = "/kaggle/input/jpx-tokyo-stock-exchange-prediction/"

# stock_prices = pd.read_csv(os.path.join(dirname, "stock_prices.csv"))

train = pd.read_csv("train.csv")



In [10]:

def make_data(df, train=0):
	data = df.set_index(["stock_id", "date_id", "seconds_in_bucket"])

	data["imbalance_buy_sell"] = data.imbalance_size * data.imbalance_buy_sell_flag

	params = ["imbalance_buy_sell", "reference_price", "matched_size", "wap"]

	data.wap = np.log(data.wap/data.wap.shift(1))

	data = data.dropna(subset=params)

	X = np.array([data.wap, data.imbalance_buy_sell]).reshape(-1,2)

	if train == 0:
		y = np.array(data.target).reshape(-1,1)
		return X, y

	return X

# def make_data(df, train=0):
# 	data = df.set_index(["stock_id", "date_id", "seconds_in_bucket"])

# 	data["imbalance_buy_sell"] = data.imbalance_size * data.imbalance_buy_sell_flag

# 	params = ["imbalance_buy_sell", "reference_price", "matched_size"]

# 	data = data.dropna(subset=params)
# 	X = np.array([data.wap, data.imbalance_buy_sell]).reshape(-1,2)

# 	if train == 0:
# 		y = np.array(data.target).reshape(-1,1)
# 		return X, y

# 	return X


In [11]:


X, y = make_data(train)

X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                   random_state=104,
                                   test_size=0.25,
                                   shuffle=True)

In [12]:
model = LinearRegression()
model.fit(X_train,y_train)


ValueError: Found input variables with inconsistent numbers of samples: [3928320, 1309440]

In [6]:
model.coef_

predict_target = model.predict(X)

In [7]:
pd.DataFrame(predict_target).describe()

,0
count,5.237760e+06
mean,-4.758886e-02
std,1.319559e-02
min,-9.534627e-01
25%,-4.773451e-02
50%,-4.773451e-02
75%,-4.773451e-02
max,1.315029e+00


In [8]:

stats.describe(predict_target)



DescribeResult(nobs=5237760, minmax=(array([-0.95346269]), array([1.31502901])), mean=array([-0.04758886]), variance=array([0.00017412]), skewness=array([6.40905745]), kurtosis=array([842.20623316]))

In [9]:
import optiver2023

env = optiver2023.make_env()
iter_test = env.iter_test()


In [10]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    if counter == 0:
        print(test.head(3))
        print(revealed_targets.head(3))
        print(sample_prediction.head(3))

    X = make_data(test, train=1)
    # sample_prediction['target'] = model.predict(X)
    sample_prediction['target'] = 0
    env.predict(sample_prediction)
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
   stock_id  date_id  seconds_in_bucket  imbalance_size  \
0         0      478                  0      3753451.43   
1         1      478                  0       985977.11   
2         2      478                  0       599128.74   

   imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                       -1         0.999875   11548975.43        NaN   
1                       -1         1.000245    3850033.97        NaN   
2                        1         1.000584    4359198.25        NaN   

   near_price  bid_price  bid_size  ask_price  ask_size  wap   row_id  
0         NaN   0.999875  22940.00   1.000050   9177.60  1.0  478_0_0  
1         NaN   0.999940   1967.90   1.000601  19692.00  1.0  478_0_1  
2         NaN   0.999918   4488.22   1.000636  34955.12  1.0  478_0_2  
   stock_id  date_id  seconds_in_bucket  revealed_target  rev

In [11]:
# for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
#     prices.head()
#     x_test = featuring_2(prices)
#     y_pred = model.predict(x_test)
#     sample_prediction['Target'] = y_pred
#     sample_prediction = sample_prediction.sort_values(by = "Target", ascending = False)
#     sample_prediction['Rank'] = np.arange(len(sample_prediction.index))
#     sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending = True)
#     sample_prediction.drop(["Target"], axis = 1)
#     submission = sample_prediction[["Date", "SecuritiesCode", "Rank"]]
#     env.predict(submission)

In [12]:
print(sample_prediction)

counter

          row_id  target
0      480_540_0       0
1      480_540_1       0
2      480_540_2       0
3      480_540_3       0
4      480_540_4       0
..           ...     ...
195  480_540_195       0
196  480_540_196       0
197  480_540_197       0
198  480_540_198       0
199  480_540_199       0

[200 rows x 2 columns]


165

In [27]:
!head submission.csv
sub = pd.read_csv("submission.csv")


row_id,target
478_0_0,0
478_0_1,0
478_0_2,0
478_0_3,0
478_0_4,0
478_0_5,0
478_0_6,0
478_0_7,0
478_0_8,0


/tmp/ipykernel_855433/676537442.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  sub = pd.read_csv("submission.csv")


In [29]:
sub[sub.target == 'target']

# type(sub.targetpd.to_numeric(df['id'], errors='coerce').notnull())
sub.target = pd.to_numeric(sub.target, errors='coerce')
sub = sub[sub.target.notnull()]


In [31]:
sub.target.max()

0.0971120314874603

In [1]:
!ls

example_test_files    optiver2023			 train.csv
__init__.py	      public_timeseries_testing_util.py
kaggle_optiver.ipynb  submission.csv
